##**Predicting the School Performance of Each Student with Lasso Regression**

Lasso regression is a statistical technique that helps select relevant characteristics to predict numerical values. It is useful when there are many characteristics and we want to identify the most influential ones on the target variable. By adding an L1 penalty to the cost function, Lasso Regression reduces coefficients of less relevant characteristics and even zeroes them.

In this project, Lasso Regression was used to predict students' grades 3 and 4 based on grades 1 and 2 and school attendance. The technique selected the most important characteristics, allowing for better-fitting models and better predictions, avoiding overfitting. The inclusion of school attendance contributed to the accuracy of the predictions, providing an informed approach to assessing student performance.

Furthermore, the provided code employs predicted assessment scores to compute the overall average, identify cases requiring remediation, calculate the final average, and determine the performance status of each student – whether they passed or failed. Fundamental parameters for analyzing each student's performance throughout the school year.

###**Library Installation**

In [ ]:
!pip install gspread

In [2]:
import gspread
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

###**Reading Spreadsheets and Converting to DataFrames**

In [4]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

planilha_treino = gc.open('Planilha 1 - Machine Learning').sheet1
dados_treino = pd.DataFrame(planilha_treino.get_all_records())

planilha_2 = gc.open('Planilha 2 - Dados para Análise').sheet1
dados_analise = pd.DataFrame(planilha_2.get_all_records())

###**Converting assessment, recovery and average columns to Float**

In [5]:
dados_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   escola nome            2200 non-null   object
 1   escola bairro          2200 non-null   object
 2   escola região          2200 non-null   object
 3   turma etapa            2200 non-null   object
 4   turma nome             2200 non-null   object
 5   componente curricular  2200 non-null   object
 6   aluno nome             2200 non-null   object
 7   aluno idade            2200 non-null   int64 
 8   aluno distorção        2200 non-null   int64 
 9   aluno etnia            2200 non-null   object
 10  avaliação 1            2200 non-null   object
 11  avaliação 2            2200 non-null   object
 12  avaliação 3            2200 non-null   object
 13  avaliação 4            2200 non-null   object
 14  média geral            2200 non-null   object
 15  recuperação          

In [6]:
# Function for converting values
def convert(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        try:
            return float(value.replace(',', '.'))
        except (ValueError, AttributeError):
            return np.nan

# List of columns that need conversion
columns_to_convert = ['avaliação 1', 'avaliação 2', 'avaliação 3', 'avaliação 4', 'média geral', 'recuperação', 'média final', 'frequência']

# Loop to convert the columns
for col in columns_to_convert:
    dados_treino[col] = dados_treino[col].apply(convert)
    dados_analise[col] = dados_analise[col].apply(convert)

In [7]:
# Replacing the voids with zero
dados_treino['recuperação'].fillna(0.0, inplace=True)
dados_analise.fillna(0.0, inplace=True)

In [8]:
#Viewing the changes made
dados_treino.head()

,escola nome,escola bairro,escola região,turma etapa,turma nome,componente curricular,aluno nome,aluno idade,aluno distorção,aluno etnia,avaliação 1,avaliação 2,avaliação 3,avaliação 4,média geral,recuperação,média final,frequência,status
0,EMEF ANISIO TEIXEIRA,ATALAIA,REGIAO 06,6° ANO,A,Geografia,ADYLLA SANTOS REIS,12,0,OUTROS,5.6,8.25,7.25,7.0,7.0,0.0,7.0,100.0,APROVADO
1,EMEF ANISIO TEIXEIRA,ATALAIA,REGIAO 06,6° ANO,A,Geografia,ALEJANDRO CELESTINO DA SILVA,12,0,OUTROS,8.5,7.00,9.50,9.0,8.5,0.0,8.5,100.0,APROVADO
2,EMEF ANISIO TEIXEIRA,ATALAIA,REGIAO 06,6° ANO,A,Geografia,ALÍCIA SILVA SANTANA,12,0,OUTROS,10.0,8.50,8.00,7.0,8.4,0.0,8.4,100.0,APROVADO
3,EMEF ANISIO TEIXEIRA,ATALAIA,REGIAO 06,6° ANO,A,Geografia,ANA CLARA SANTOS DE JESUS,12,0,OUTROS,5.1,6.25,5.50,5.0,5.5,0.0,5.5,100.0,APROVADO
4,EMEF ANISIO TEIXEIRA,ATALAIA,REGIAO 06,6° ANO,A,Geografia,BYANKA ANNEMBERG SANTOS NOGUEIRA,12,0,OUTROS,2.0,2.75,3.00,4.0,2.9,5.0,5.0,100.0,APROVADO


In [9]:
#Viewing the changes made
dados_analise.head()

,escola nome,escola bairro,escola região,turma etapa,turma nome,componente curricular,aluno nome,aluno idade,aluno distorção,aluno etnia,avaliação 1,avaliação 2,avaliação 3,avaliação 4,média geral,recuperação,média final,frequência,status
0,Escola X,Bairro A,4º Região,2º Ano,Turma 1,Matemática,Ana Souza,8,0,Branco,7.8,8.2,0.0,0.0,0.0,0.0,0.0,85.0,
1,Escola Y,Bairro B,1º Região,1º Ano,Turma 2,Língua Portuguesa,João Pereira,7,0,Amarelo,8.0,7.5,0.0,0.0,0.0,0.0,0.0,88.0,
2,Escola Z,Bairro C,3º Região,5º Ano,Turma 3,Geografia,Sophia Silva,11,0,Pardo,6.8,7.2,0.0,0.0,0.0,0.0,0.0,78.0,
3,Escola W,Bairro D,6º Região,8º Ano,Turma 4,Ciências da Natureza,Lucas Souza,14,0,Branco,7.5,8.0,0.0,0.0,0.0,0.0,0.0,85.0,
4,Escola V,Bairro E,2º Região,3º Ano,Turma 5,História,Maria Oliveira,9,0,Pardo,8.5,8.8,0.0,0.0,0.0,0.0,0.0,90.0,


###**Criando e Treinando o Modelo**

In [10]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Define X and the target sets (y) for each evaluation and recovery
X = dados_treino[['avaliação 1', 'avaliação 2','frequência']]

y_avaliacao3 = dados_treino['avaliação 3']
y_avaliacao4 = dados_treino['avaliação 4']
y_recuperacao = dados_treino['recuperação']

# Divide the data into training and test
X_train, X_test, y_train_avaliacao3, y_test_avaliacao3 = train_test_split(X, y_avaliacao3, test_size=0.2, random_state=42)
_, _, y_train_avaliacao4, y_test_avaliacao4 = train_test_split(X, y_avaliacao4, test_size=0.2, random_state=42)
_, _, y_train_recuperacao, y_test_recuperacao = train_test_split(X, y_recuperacao, test_size=0.2, random_state=42)

from sklearn.model_selection import GridSearchCV

# Alpha values to test
alphas = [0.001, 0.01, 0.1, 1.0, 10.0]

# Parameters for grid search
param_grid = {'alpha': alphas}

# Create a GridSearchCV object for Assessment 3
grid_search_avaliacao3 = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_avaliacao3.fit(X_train, y_train_avaliacao3)

# Best alpha value found
best_alpha_avaliacao3 = grid_search_avaliacao3.best_params_['alpha']

# Train the model with the best alpha value
model_avaliacao3 = Lasso(alpha=best_alpha_avaliacao3)
model_avaliacao3.fit(X_train, y_train_avaliacao3)

# Make predictions for Assessment 3 in the test set
y_pred_avaliacao3 = model_avaliacao3.predict(X_test)

In [11]:
# Parameters for grid search
param_grid = {'alpha': alphas}

# Create a GridSearchCV object for Evaluation 4
grid_search_avaliacao4 = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_avaliacao4.fit(X_train, y_train_avaliacao4)

# Best alpha value found
best_alpha_avaliacao4 = grid_search_avaliacao4.best_params_['alpha']

# Lasso Regression model training for Evaluation 4
model_avaliacao4 = Lasso(alpha=best_alpha_avaliacao4)
model_avaliacao4.fit(X_train, y_train_avaliacao4)

# Prediction for Assessment 4 on test data
y_pred_avaliacao4 = model_avaliacao4.predict(X_test)

# Parameters for grid search
param_grid = {'alpha': alphas}

In [12]:
# Create a GridSearchCV object for Recovery
grid_search_recuperacao = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_recuperacao.fit(X_train, y_train_recuperacao)

# Best alpha value found
best_alpha_recuperacao = grid_search_recuperacao.best_params_['alpha']

# Training the Lasso Regression model for Recovery
model_recuperacao = Lasso(alpha=best_alpha_recuperacao)
model_recuperacao.fit(X_train, y_train_recuperacao)

# Forecast for Recovery in test data
y_pred_recuperacao = model_recuperacao.predict(X_test)

###**Defining important functions for filling in Spreadsheet 3**

In [13]:
# Function to calculate the overall average
def calcular_media(row):
    notas = [row['avaliação 1'], row['avaliação 2'], row['avaliação 3'], row['avaliação 4']]
    notas = [nota for nota in notas if nota != 0.0]
    if len(notas) == 0:
        return None
    return sum(notas) / len(notas)

# Function to determine status
def determinar_status(row):
    if row['média final'] >= 5 and row['frequência'] >= 75.0:
        return 'APROVADO'
    else:
        return 'REPROVADO'

def preencher_valores_vazios(row):
    if row['avaliação 3'] == 0.0 or pd.isnull(row['avaliação 3']):
        X_predict = row[['avaliação 1', 'avaliação 2', 'frequência']].values.reshape(1, -1)
        predicted_nota3 = model_avaliacao3.predict(X_predict)
        row['avaliação 3'] = predicted_nota3[0]

    if row['avaliação 4'] == 0.0 or pd.isnull(row['avaliação 4']):
        X_predict = row[['avaliação 1', 'avaliação 2', 'frequência']].values.reshape(1, -1)
        predicted_nota4 = model_avaliacao4.predict(X_predict)
        row['avaliação 4'] = predicted_nota4[0]

    row['média geral'] = calcular_media(row)

    if row['avaliação 3'] != 0.0 and row['avaliação 4'] != 0.0:
        if row['média geral'] < 5 and (row['recuperação'] == 0.0 or pd.isnull(row['recuperação'])):
            X_predict = row[['avaliação 1', 'avaliação 2', 'frequência']].values.reshape(1, -1)
            predicted_recup = model_recuperacao.predict(X_predict)
            row['recuperação'] = predicted_recup[0]

    if row['média geral'] < 5:
        row['média final'] = max(row['recuperação'], row['média geral'])
    else:
        row['média final'] = row['média geral']

    return row

###**Exporting the data to Spreadsheet 3**

In [ ]:
# Creating the result dataframe
resultado = dados_analise.apply(preencher_valores_vazios, axis=1)
resultado['média geral'] = resultado.apply(calcular_media, axis=1)
resultado['média final'] = resultado.apply(lambda row: max(row['recuperação'], row['média geral']) if row['média geral'] < 5 else row['média geral'], axis=1)
resultado['status'] = resultado.apply(determinar_status, axis=1)

In [20]:
# Replace the values 0.0 with NaN in the recovery column
resultado['recuperação'] = resultado['recuperação'].replace(0.0, float('NaN'))

In [25]:
# Substituir NaN por uma string representativa
resultado = resultado.fillna("NA")

In [26]:
#Viewing the result dataframe
print(resultado)

   escola nome escola bairro escola região turma etapa turma nome  \
0     Escola X      Bairro A     4º Região      2º Ano    Turma 1   
1     Escola Y      Bairro B     1º Região      1º Ano    Turma 2   
2     Escola Z      Bairro C     3º Região      5º Ano    Turma 3   
3     Escola W      Bairro D     6º Região      8º Ano    Turma 4   
4     Escola V      Bairro E     2º Região      3º Ano    Turma 5   
5     Escola U      Bairro F     7º Região      9º Ano    Turma 6   
6     Escola T      Bairro G     5º Região      6º Ano    Turma 7   
7     Escola S      Bairro H     8º Região      8º Ano    Turma 8   
8     Escola R      Bairro I     2º Região      2º Ano    Turma 9   
9     Escola Q      Bairro J     1º Região      1º Ano   Turma 10   
10    Escola P      Bairro K     7º Região      9º Ano   Turma 11   
11    Escola O      Bairro L     5º Região      6º Ano   Turma 12   
12    Escola N      Bairro M     4º Região      2º Ano   Turma 13   
13    Escola M      Bairro N     3

In [27]:
# Count the number of successful and unsuccessful students
count_status = resultado['status'].value_counts()

# Show the result
print("Quantidade de Aprovados:", count_status['APROVADO'])
print("Quantidade de Reprovados:", count_status['REPROVADO'])

Quantidade de Aprovados: 24
Quantidade de Reprovados: 6


In [28]:
# Creating and exporting data to Spreadsheet 3

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Create a new spreadsheet
spreadsheet = gc.create('Planilha 3 - Resultado')

# Get the first sheet of the new spreadsheet
worksheet = spreadsheet.get_worksheet(0)

# Turn the resulting DataFrame into a list of lists
dados_lista = [resultado.columns.tolist()] + resultado.values.tolist()

# Defining the range where the data will be added (starting from cell A1)
range_start = 'A1'
range_end = chr(64 + len(resultado.columns)) + str(len(resultado) + 1)

# Sending the data to worksheet 3
worksheet.update(range_start + ':' + range_end, dados_lista)

print("DataFrame Result successfully added to Google Sheets!")

DataFrame Result successfully added to Google Sheets!
